In [15]:
import parsy as P
dec=P.regex('[+-]?[0-9]+').map(int).desc('decimal integer')
none=P.string('').result(None)
dec_none=(dec|none).map(lambda x: None if x is None else int(x)).desc('possibly-empty-decimal')
colon=P.string(':')
comma=P.string(',')
dot=P.string('.')
lbrack,rbrack=P.string('['),P.string(']')
pipe=P.string('|')
# print(multiindex.parse(','))

balanced=P.forward_declaration()
bracketed=P.seq(lbrack,balanced,rbrack).concat()
plain=P.regex('[^[\]]+')
balanced.become((plain|bracketed).many().concat().desc('balanced-expression'))

index=dec.map(int).tag('index')
#multiindex=((dec.sep_by(comma,min=1))<<(none|comma)).tag('multiindex')
multiindex=((dec<<comma).times(1,float('inf'))+dec.times(0,1)).tag('multiindex')
slice=(dec_none).sep_by(colon,min=2,max=3).tag('slice')
filter=((pipe>>balanced)|none).tag('filter')
subscript=(lbrack>>P.seq(slice|multiindex|index,filter)<<rbrack).combine_dict(dict)

attr=P.regex('[a-zA-Z_][a-zA-Z_0-9]*')

def mk_dotpath(vv):
    return [{'attr':v['attr']}|(dict(v['sub'] if v['sub'] is not None else {})) for v in vv]


dotpath=P.seq(attr=attr,sub=subscript|none).sep_by(dot).map(mk_dotpath)
# expr_py.parse('sdfsd[1232][23423]')

In [16]:
#multiindex.parse('13,')
#slice.parse('::2')
for s in ['[1232]','[]','[1,]','[,]','[:]','[::]','[1:2:3]','[::-1]','[::|F]','[1,2]']:
    try: p=subscript.parse(s)
    except P.ParseError as e: p=f'ERROR: {str(e)}'
    print(f'{s} → {p}')

[1232] → {'index': 1232, 'filter': None}
[] → ERROR: expected one of ':', 'decimal integer' at 0:1
[1,] → {'multiindex': [1], 'filter': None}
[,] → ERROR: expected one of ':', 'decimal integer' at 0:1
[:] → {'slice': [None, None], 'filter': None}
[::] → {'slice': [None, None, None], 'filter': None}
[1:2:3] → {'slice': [1, 2, 3], 'filter': None}
[::-1] → {'slice': [None, None, -1], 'filter': None}
[::|F] → {'slice': [None, None, None], 'filter': 'F'}
[1,2] → {'multiindex': [1, 2], 'filter': None}


In [17]:
# dotpath.parse('la[1].bbb[2|sdfsd[3]]')
#dotpath.parse('aa')
dotpath.parse('aa[1]')
#dotpath.parse('aa[1,2,3|f(ryx).ssd[12]].bb[::|a<44]')

[{'attr': 'aa', 'index': 1, 'filter': None}]

In [9]:
import re
re.match('[^][]+','asdas[')

<re.Match object; span=(0, 5), match='asdas'>